## Parte 1 ● Ejecutar los scripts con una muestra relativamente pequeña de cada uno de losendpoints y verificar que todo funcione correctamente.

○ Demanda 

○ Balance

https://www.ree.es/es/datos/apidatos




In [2]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint

In [3]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

numpy==1.26.4
pandas==2.2.2
requests==2.32.3


In [ ]:
# Endpoint
url = "https://apidatos.ree.es/"

# Parámetros

"""
    ### He visto que en los parametros no pueden ir los dos, geo_limit y geo_id juntos
        asi que tenemos que pensar como hacer que los parametros cambien entre geo_limit
        y geo_id para que no coincidan los dos a la vez.

    ### Tambien hay que pensar como hacer que se coja el valor de la region, algo asi como que
        si geo_limit = 4 entonces region = "Andalucia" y si geo_limit = 5 entonces region = "Aragon"

    ### En el caso de que geo_limit = "peninsular" entonces no se tiene que poner el geo_id

    ### En el caso de que geo_limit = "baleares" entonces no se tiene que poner el geo_id

    ### En el caso de que geo_limit = "canarias" entonces no se tiene que poner el geo_id

    ### En el caso de que geo_limit = "ceuta" entonces no se tiene que poner el geo_id

    ### En el caso de que geo_limit = "ccaa" entonces no se tiene que poner el geo_id

"""
params = {
    "start_date": "2019-01-01T00:00",
    "end_date": "2019-12-31T23:59",
    "time_trunc": "day",
    #"geo_limit":#"peninsular"
    "geo_limit":"canarias"
    #"geo_limit":"baleares"
    #"geo_limit":"ceuta"
    #"geo_limit":"ccaa"
    #apartir de aqui se incluye el geo_id para las demas regiones ccaa
    #"geo_id": "4" #Andalucía
    #"geo_id": "5" #Aragón
    #"geo_id": "6" #Cantabria
    #"geo_id": "7" #castilla la mancha
    #"geo_id": "8" #castilla y león
    #"geo_id": "9" #cataluña
    #"geo_id": "10" #pais vasco
    #"geo_id": "11" #principado de asturias
    #"geo_id": "13" #comunidada de madrid
    #"geo_id": "14" #comunidada de navarra
    #"geo_id": "15" #comunidada de valenciana
    #"geo_id": "16" #extremadura
    #"geo_id": "17" #galicia
    #"geo_id": "20" #la rioja
    #"geo_id": "21" #region de murcia
}

# Headers + User-Agent
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
    "User-Agent": "Mozilla/5.0"
}

In [ ]:
### ✅ BALANCE ###

filtro_balance = 'es/datos/balance/balance-electrico'

endpoint = f'{url}{filtro_balance}'

#request
response = requests.get(endpoint, headers=headers, params=params)

data = response.json()

included = data['included']

registros = []

for grupo in included:
    group_name = grupo['attributes']['title']  # Ej: "Renovable"
    contenidos = grupo['attributes'].get('content', [])

    for item in contenidos:
        indicador = item['attributes']['title']  # Ej: "Hidráulica"
        valores = item['attributes'].get('values', [])

        for punto in valores:
            registros.append({
                'datetime': punto['datetime'],
                'value': punto['value'],
                'indicator': indicador,
                'group': group_name,
                'region': 'canarias'
            })

df_balance = pd.DataFrame(registros)

print(df_balance.head())

df_balance.to_csv('../data/sinprocesar/balance_electrico_dia_2019_canarias.csv', index=False)

                        datetime      value   indicator      group    region
0  2019-01-01T00:00:00.000+01:00  53722.120  Hidráulica  Renovable  canarias
1  2019-01-02T00:00:00.000+01:00  61294.768  Hidráulica  Renovable  canarias
2  2019-01-03T00:00:00.000+01:00  82981.951  Hidráulica  Renovable  canarias
3  2019-01-04T00:00:00.000+01:00  94301.940  Hidráulica  Renovable  canarias
4  2019-01-05T00:00:00.000+01:00  58822.692  Hidráulica  Renovable  canarias


GENERACION E INTERCAMBIO

In [27]:
### ✅ GENERACION ###

filtro_generacion = "/es/datos/generacion/estructura-generacion"
endpoint = f"{url}{filtro_generacion}"

response = requests.get(endpoint, headers=headers, params=params)

if response.status_code == 200:
    try:
        data = response.json()

        included = data.get('included', [])

        if included:
            registros = []

            # Iterar sobre los grupos de datos en 'included'
            for grupo in included:
                group_name = grupo['attributes'].get('title', 'Desconocido')
                group_type = grupo['attributes'].get('type', 'Desconocido')
                contenidos = grupo['attributes'].get('values', [])

                # Iterar sobre los valores de generación
                for punto in contenidos:
                    registros.append({
                        'datetime': punto.get('datetime', 'Desconocido'),
                        'value': punto.get('value', 0),
                        'percentage': punto.get('percentage', 0),
                        'indicator': group_name,
                        'region': 'canarias',
                        'Type': group_type,
                    })

            df_generacion = pd.DataFrame(registros)

            print('Datos de generación:')
            display(df_generacion)

            df_generacion.to_csv('../data/sinprocesar/generacion_estructura_2019_peninsular.csv', index=False)


    except:
        print("Contenido de la respuesta:", response.text)

else:
    print(f"Error en la solicitud. Código: {response.status_code}")

Datos de generación:


,datetime,value,percentage,indicator,region,Type
0,2019-01-01T00:00:00.000+01:00,53722.120,0.097735,Hidráulica,canarias,Renovable
1,2019-01-02T00:00:00.000+01:00,61294.768,0.086331,Hidráulica,canarias,Renovable
2,2019-01-03T00:00:00.000+01:00,82981.951,0.115989,Hidráulica,canarias,Renovable
3,2019-01-04T00:00:00.000+01:00,94301.940,0.125121,Hidráulica,canarias,Renovable
4,2019-01-05T00:00:00.000+01:00,58822.692,0.086633,Hidráulica,canarias,Renovable
...,...,...,...,...,...,...
5833,2019-12-27T00:00:00.000+01:00,653199.592,1.000000,Generación total,canarias,Generación total
5834,2019-12-28T00:00:00.000+01:00,636182.923,1.000000,Generación total,canarias,Generación total
5835,2019-12-29T00:00:00.000+01:00,618471.120,1.000000,Generación total,canarias,Generación total
5836,2019-12-30T00:00:00.000+01:00,641123.107,1.000000,Generación total,canarias,Generación total


In [ ]:
### ✅ INTERCAMBIO ###

# Lista de países
lista_paises = ['francia-frontera', 'portugal-frontera', 'marruecos-frontera', 'andorra-frontera']

datos_intercambios = []

for pais in lista_paises:
    print(f'Consultando datos de {pais}...')

    filtro_intercambio = f"/es/datos/intercambios/{pais}"
    endpoint = f"{url}{filtro_intercambio}"

    response = requests.get(endpoint, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()

        valores = data["included"][0]["attributes"].get("values", [])
        if valores:
            df = pd.DataFrame(valores)
            df["pais"] = pais
            datos_intercambios.append(df)
    else:
        print(f"Error en la solicitud para {pais}. Código de estado: {response.status_code}")

df_intercambio = pd.concat(datos_intercambios, ignore_index=True)

print(f"Datos de intercambios con {len(datos_intercambios)} países:")
display(df_intercambio)

df_intercambio.to_csv('../data/sinprocesar/intercambios_2019_canarias.csv', index=False)

Consultando datos de francia-frontera...
Consultando datos de portugal-frontera...
Consultando datos de marruecos-frontera...
Consultando datos de andorra-frontera...
Datos de intercambios con 4 países:


,value,percentage,datetime,pais
0,-208.239,0.003465,2019-01-01T00:00:00.000+01:00,francia-frontera
1,-2325.553,0.036857,2019-01-02T00:00:00.000+01:00,francia-frontera
2,-11123.719,0.243767,2019-01-03T00:00:00.000+01:00,francia-frontera
3,-17758.093,0.397978,2019-01-04T00:00:00.000+01:00,francia-frontera
4,-4267.317,0.102386,2019-01-05T00:00:00.000+01:00,francia-frontera
...,...,...,...,...
1455,-434.390,1.000000,2019-12-27T00:00:00.000+01:00,andorra-frontera
1456,-796.650,1.000000,2019-12-28T00:00:00.000+01:00,andorra-frontera
1457,-728.240,1.000000,2019-12-29T00:00:00.000+01:00,andorra-frontera
1458,-405.490,1.000000,2019-12-30T00:00:00.000+01:00,andorra-frontera


In [263]:
################################################################################################################################